<a href="https://colab.research.google.com/github/alishawahna9/Flora-Vision/blob/main/Copy_of_bashar_final_prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-firebase

  Preparing metadata (setup.py) ... done
  Created wheel for python-firebase: filename=python_firebase-1.2-py3-none-any.whl size=11514 sha256=a2d98c71fe0f4e16f450ecb6c869c347d1ca5193052801efcff56b5a2219154c
  Stored in directory: /root/.cache/pip/wheels/df/1e/df/086655a94205163cc541d2e63d251987b045edd56fbb535150
Successfully built python-firebase


In [2]:
import requests
import json
from datetime import datetime

FIREBASE_DB_URL = "https://prototype-48118-default-rtdb.firebaseio.com/"

def fetch_sensor_history(limit=24):
    """
    שולף את ה-limit רשומות האחרונות מ-Firebase
    ומחזיר רשימות מוכנות לגרפים.
    """
    try:
        # שליפה ממוינת לפי מפתח (זמן) ומוגבלת לכמות האחרונה
        url = f"{FIREBASE_DB_URL}/sensors/history.json"
        params = {
            "orderBy": '"$key"',
            "limitToLast": limit
        }
        r = requests.get(url, params=params, timeout=10)
        r.raise_for_status()
        data = r.json()

        if not data:
            return None

        # המרת המילון לרשימה ממוינת
        sorted_keys = sorted(data.keys())

        # יצירת רשימות התוצאה
        temps = []
        hums = []
        soils = []
        times = []

        for key in sorted_keys:
            entry = data[key]
            # חילוץ נתונים (עם הגנה מפני ערכים חסרים)
            t_val = float(entry.get("temperature", 0))
            h_val = float(entry.get("humidity", 0))
            s_val = float(entry.get("soil", 0))

            # ניסיון חילוץ שעה מהמפתח (שהוא בפורמט תאריך)
            # הפורמט הוא: YYYY-MM-DD_HH-MM-SS-ffffff
            try:
                time_str = key.split("_")[1][:5].replace("-", ":") # HH:MM
            except:
                time_str = "00:00"

            temps.append(t_val)
            hums.append(h_val)
            soils.append(s_val)
            times.append(time_str)

        return {
            "temperature": temps,
            "humidity": hums,
            "soil": soils,
            "labels": times
        }

    except Exception as e:
        print(f"Error fetching history: {e}")
        return None

In [3]:
knkFIREBASE_URL = "https://prototype-48118-default-rtdb.firebaseio.com/"

In [4]:
# =========================
# BLOCK 1: BACKEND LOGIC & MODELS (FIXED)
# Run this ONCE. It loads the AI models into memory.
# =========================

import io
from PIL import Image
from transformers import pipeline

print("Initializing Botanical Systems...")

# --- optional accel (safe) ---
try:
    import torch
    _HAS_TORCH = True
except:
    _HAS_TORCH = False

def _best_device():
    # transformers pipeline accepts: device=-1 (cpu) or 0.. for cuda
    if _HAS_TORCH and torch.cuda.is_available():
        return 0
    return -1

_DEVICE = _best_device()

# 1) Load models only once
try:
    if 'identity_classifier' not in globals():
        print("Loading Identity Model...")
        identity_classifier = pipeline(
            "image-classification",
            model="umutbozdag/plant-identity",
            device=_DEVICE
        )

    if 'disease_classifier' not in globals():
        print("Loading Disease Model...")
        disease_classifier = pipeline(
            "image-classification",
            model="Diginsa/Plant-Disease-Detection-Project",
            device=_DEVICE
        )

    if 'health_classifier' not in globals():
        print("Loading Health Model...")
        health_classifier = pipeline(
            "image-classification",
            model="swueste/plant-health-image-classifier",
            device=_DEVICE
        )

    if 'advice_model' not in globals():
        print("Loading Advice Model (LLM)...")
        # flan-t5-large can be heavy; device helps if GPU exists
        advice_model = pipeline(
            "text2text-generation",
            model="google/flan-t5-large",
            device=_DEVICE
        )

    print("✅ All Models Loaded Successfully. (device:", "GPU" if _DEVICE == 0 else "CPU", ")")

except Exception as e:
    print(f"⚠️ Error loading models: {e}")

# 2) Gardening advice (LLM)
def get_botanical_advice(plant_name, health_status=None):
    prompt = (
        f"Provide accurate gardening facts for the plant '{plant_name}'. "
        f"Specify: 1. Sunlight needs (Full Sun / Shade). "
        f"2. Watering frequency (High / Low). "
        f"3. Soil type."
    )
    try:
        result = advice_model(
            prompt,
            max_length=160,
            do_sample=False,       # more stable than sampling
            num_beams=3
        )
        return (result[0].get('generated_text') or "").strip() or "No advice returned."
    except Exception:
        return "Could not retrieve specific advice at this moment."

# 2.5) AI helper for RAG (replaces Gemini)
def rag_ai_answer(question: str, docs_text: str):
    """
    Use flan-t5-large to answer a question using ONLY provided context.
    docs_text should be short (a few paragraphs).
    """
    prompt = (
        "Answer the question using ONLY the context. "
        "If the answer is not in the context, say: 'I don't know based on the provided sources.'\n\n"
        f"Question: {question}\n\n"
        f"Context:\n{docs_text}\n"
    )
    try:
        out = advice_model(prompt, max_length=220, do_sample=False, num_beams=3)
        return (out[0].get("generated_text") or "").strip()
    except Exception:
        return "AI is currently unavailable."

# 3) Main image analysis logic
def run_analysis_logic(image_bytes: bytes):
    # safer image handling
    image = Image.open(io.BytesIO(image_bytes)).convert("RGB")

    # run models
    id_res = identity_classifier(image)[0]
    health_res = health_classifier(image)[0]
    disease_raw = disease_classifier(image)

    # parse
    plant_name = id_res.get('label', 'Unknown')
    disease_list = [
        f"{res.get('label','?')} ({float(res.get('score',0)):.1%})"
        for res in (disease_raw or [])[:2]
    ]

    advice = get_botanical_advice(plant_name, health_res.get('label'))

    return {
        "name": plant_name,
        "id_conf": f"{float(id_res.get('score',0)):.1%}",
        "health": health_res.get('label', 'Unknown'),
        "health_conf": f"{float(health_res.get('score',0)):.1%}",
        "diseases": disease_list if disease_list else ["No disease signals detected."],
        "advice": advice
    }


Initializing Botanical Systems...
Loading Identity Model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/327 [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


✅ All Models Loaded Successfully. (device: CPU )


In [5]:
# --- 2. BUILD UI COMPONENTS (FIXED) ---
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display
import re

style_html_upload = widgets.HTML("""
<style>
/* ===== Results text fix ===== */
.fv-upload-scope .res-title,
.fv-upload-scope .ai-pill,
.fv-upload-scope .ai-advice-box,
.fv-upload-scope div,
.fv-upload-scope span,
.fv-upload-scope p {
  color: #111827 !important;
}

/* ===== cards ===== */
.fv-upload-scope .flora-card {
  background: #ffffff;
  border-radius: 28px;
  box-shadow: 0 16px 40px rgba(15,23,42,0.08);
}

.fv-upload-scope .flora-tips {
  background: linear-gradient(90deg, #f4fbf3 0%, #f4fbff 55%, #f5f7ff 100%);
  border-radius: 24px;
  border: 1px solid #d7e3ff;
  box-shadow: 0 10px 26px rgba(15,23,42,0.05);
}

/* ===== dropzone ===== */
.fv-upload-scope .flora-dropzone {
  position: relative !important;
  overflow: hidden;
  border: 2px dashed #d4dde9 !important;
  border-radius: 32px !important;
  transition: background .15s ease, box-shadow .15s ease, border-color .15s ease;
}

.fv-upload-scope .flora-dropzone:hover {
  background: radial-gradient(
    circle at center,
    rgba(34,197,94,.12) 0%,
    rgba(34,197,94,.06) 42%,
    rgba(255,255,255,.96) 100%
  ) !important;
  border-color: rgba(34,197,94,.45) !important;
  box-shadow:
    0 0 0 2px rgba(34,197,94,.18) inset,
    0 18px 34px rgba(15,23,42,.08);
}

/* ===== FileUpload overlay ===== */
.fv-upload-scope .flora-upload-btn {
  position: absolute !important;
  inset: 0 !important;
  width: 100% !important;
  height: 100% !important;
  z-index: 50 !important;
  background: transparent !important;
  border: none !important;
}

/* hide all internal text */
.fv-upload-scope .flora-upload-btn,
.fv-upload-scope .flora-upload-btn * {
  font-size: 0 !important;
  color: transparent !important;
}

/* ===== Analyze Button ===== */
.fv-upload-scope .analyze-btn-style {
  background: linear-gradient(180deg, #22c55e 0%, #16a34a 100%) !important;
  color: #ffffff !important;
  font-weight: 800 !important;
  letter-spacing: 0.04em;
  border: none !important;
  border-radius: 14px !important;
  box-shadow: 0 10px 22px rgba(34,197,94,.35) !important;
}

.fv-upload-scope .analyze-btn-style:hover {
  background: linear-gradient(180deg, #16a34a 0%, #15803d 100%) !important;
  box-shadow: 0 14px 28px rgba(34,197,94,.45) !important;
}

.fv-upload-scope .analyze-btn-style:disabled {
  opacity: 0.65 !important;
  cursor: not-allowed !important;
  transform: none !important;
  filter: none !important;
}

/* ===== Reset Button ===== */
.fv-upload-scope .reset-btn-style {
  background: #ffffff !important;
  color: #374151 !important;
  border: 1px solid #e5e7eb !important;
  border-radius: 999px !important;
  padding: 8px 16px !important;
  font-size: 13px !important;
}
.fv-upload-scope .reset-btn-style:hover {
  background: #f8fafc !important;
  border-color: #cbd5e1 !important;
}

/* ===== AI Report Card ===== */
.ai-report {
  margin-top: 28px;
  padding: 26px 28px;
  background: linear-gradient(180deg, #ffffff 0%, #f8fafc 100%);
  border-radius: 20px;
  border: 1px solid #e5e7eb;
  box-shadow: 0 18px 40px rgba(15,23,42,.08);
}

.ai-header { display:flex; align-items:center; gap:14px; margin-bottom:18px; }
.ai-header-icon { font-size: 34px; }
.ai-title { font-size: 16px; font-weight:800; color:#111827; }
.ai-subtitle { font-size: 12px; color:#6b7280; }

.ai-pills { display:flex; gap:10px; margin-bottom:22px; flex-wrap:wrap; }
.ai-pill { padding:6px 14px; border-radius:999px; font-size:12px; font-weight:700; }
.ai-pill.plant { background:#ecfdf5; color:#065f46; }
.ai-pill.health { background:#fef2f2; color:#991b1b; }

.ai-section { margin-bottom:18px; }
.ai-section-title { font-size:13px; font-weight:800; margin-bottom:6px; color:#374151; }

.ai-symptoms {
  background:#f8fafc;
  border-left:4px solid #f97316;
  padding:12px 14px;
  border-radius:8px;
  font-size:13px;
  color:#374151;
}

.ai-advice {
  background: linear-gradient(180deg, #ecfdf5 0%, #f0fdf4 100%);
  border-left: 4px solid #22c55e;
  padding: 14px 16px;
  border-radius: 10px;
  font-size: 13px;
  color: #065f46;
  line-height: 1.6;
}
</style>
""")

PRIMARY    = "#22c55e"
TEXT_MUTED = "#64748b"

# Titles
card_title = widgets.HTML("<div style='font-size:24px;font-weight:900;color:#0f172a;margin-bottom:4px;text-align:center;'>FloraVision AI</div>")
card_subtitle = widgets.HTML(f"<div style='font-size:14px;color:{TEXT_MUTED};margin-bottom:28px;text-align:center;'>Advanced Plant Health Diagnostics</div>")

# Upload Area (Visuals - what you SEE)
upload_icon = widgets.HTML(f"""
<div style="width:80px;height:80px;border-radius:50%;background:#e9fbea;display:flex;align-items:center;justify-content:center;margin-bottom:12px;box-shadow:0 8px 16px rgba(34,197,94,0.1);">
  <span style="font-size:36px;color:{PRIMARY};">⬆</span>
</div>
""")

upload_text = widgets.VBox([
    widgets.HTML("<div style='font-size:16px;font-weight:700;color:#1e293b;margin-bottom:4px;text-align:center;'>Click to Upload Photo</div>"),
    widgets.HTML(f"<div style='font-size:12px;color:{TEXT_MUTED};text-align:center;'>JPG, PNG, HEIC (Max 10MB)</div>")
])

upload_inner = widgets.VBox(
    [upload_icon, upload_text],
    layout=Layout(align_items="center", justify_content="center", width="100%", min_height="260px")
)

# File uploader (overlay)
file_uploader = widgets.FileUpload(accept=".jpg,.jpeg,.png,.heic", multiple=False)
file_uploader.add_class("flora-upload-btn")

# Dropzone container
upload_box = widgets.Box(
    [upload_inner, file_uploader],
    layout=Layout(
        width="100%",
        padding="40px 32px",
        border="none",
        border_radius="32px",
        justify_content="center",
        align_items="center",
        position="relative"
    )
)
upload_box.add_class("flora-dropzone")

# Preview & Actions
img_preview = widgets.Image(
    layout=Layout(
        height="300px",
        width="100%",
        object_fit="contain",
        margin="0 0 20px 0",
        border_radius="12px"
    )
)

btn_analyze = widgets.Button(
    description="RUN ANALYSIS",
    layout=Layout(width="100%", height="56px")
)
btn_analyze.add_class("analyze-btn-style")

btn_reset = widgets.Button(
    description="Upload Different Image",
    icon="refresh",
    layout=Layout(width="auto", margin="16px 0 0 0")
)
btn_reset.add_class("reset-btn-style")

preview_container = widgets.VBox(
    [img_preview, btn_analyze, widgets.Box([btn_reset], layout=Layout(justify_content="center"))],
    layout=Layout(display="none", width="100%", align_items="center")
)

out_results = widgets.Output()

main_card_content = widgets.VBox(
    [card_title, card_subtitle, upload_box, preview_container, out_results],
    layout=Layout(width="70%", padding="40px 40px 32px 40px", margin="24px auto 18px auto")
)
main_card_content.add_class("flora-card")

# Tips Section
def tip(color, title, desc, icon="📷"):
    return widgets.HBox([
        widgets.HTML(f"""<div style="width:40px;height:40px;border-radius:12px;background:{color};display:flex;align-items:center;justify-content:center;margin-right:12px;"><span style="font-size:20px;color:white;">{icon}</span></div>"""),
        widgets.VBox([
            widgets.HTML(f"<div style='font-size:13px;font-weight:700;color:#1e293b;margin-bottom:2px;'>{title}</div>"),
            widgets.HTML(f"<div style='font-size:11px;color:{TEXT_MUTED};'>{desc}</div>")
        ]),
    ], layout=Layout(width="50%", padding="10px 0"))

tips_card = widgets.VBox(
    [
        widgets.HTML("<div style='font-size:12px;font-weight:800;color:#94a3b8;text-transform:uppercase;margin-bottom:15px;letter-spacing:1px;'>Photography Guide</div>"),
        widgets.HBox(
            [tip("#22c55e", "Natural Light", "Avoid dark shadows", "☀️"),
             tip("#3b82f6", "Focus", "Keep 20cm distance", "🔎")],
            layout=Layout(justify_content="space-between")
        ),
        widgets.HBox(
            [tip("#a855f7", "Leaf Detail", "Capture spots clearly", "🌿"),
             tip("#f97316", "Background", "Use neutral colors", "⬜")],
            layout=Layout(justify_content="space-between")
        )
    ],
    layout=Layout(width="70%", padding="18px 26px 22px 26px", margin="0 auto 32px auto")
)
tips_card.add_class("flora-tips")

# -------------------------
# Event Handlers (FIXED)
# -------------------------
def _get_uploaded_content(uploader):
    vals = uploader.value
    if isinstance(vals, tuple) and vals:
        return vals[0].get("content", b"")
    if isinstance(vals, dict) and vals:
        return vals[list(vals.keys())[0]].get("content", b"")
    return b""

def _clear_uploader(u):
    """
    Robust reset for FileUpload in Colab (tuple/dict variants).
    Also allows re-uploading the SAME filename again.
    """
    # dict variant
    try:
        u.value.clear()
    except:
        pass
    # force reset
    for v in ({}, (), None):
        try:
            u.value = v
        except:
            pass
    # reset internal counter (some versions)
    for attr in ("_counter",):
        if hasattr(u, attr):
            try:
                setattr(u, attr, 0)
            except:
                pass

def on_upload_change(change):
    content = _get_uploaded_content(file_uploader)
    if not content:
        return
    img_preview.value = content
    upload_box.layout.display = "none"
    preview_container.layout.display = "flex"
    out_results.clear_output()

def on_reset_click(_):
    _clear_uploader(file_uploader)
    img_preview.value = b""
    upload_box.layout.display = "flex"
    preview_container.layout.display = "none"
    out_results.clear_output()
    # חשוב: מאפשר להעלות שוב את אותו קובץ בלי "להיתקע"
    try:
        file_uploader._dom_classes = file_uploader._dom_classes  # no-op to trigger refresh in some cases
    except:
        pass

def on_analyze_click(_):
    if "run_analysis_logic" not in globals():
        with out_results:
            out_results.clear_output()
            print("❌ Error: Logic Block 1 is missing. Please run it first.")
        return

    if not img_preview.value:
        with out_results:
            out_results.clear_output()
            print("❌ Please upload an image first.")
        return

    # lock button to prevent double-clicks
    btn_analyze.disabled = True
    btn_analyze.description = "ANALYZING..."

    with out_results:
        out_results.clear_output()
        display(widgets.HTML(
            "<div style='text-align:center; padding:20px; color:#64748b;'>🔄 Processing AI Models...</div>"
        ))

    try:
        results = run_analysis_logic(img_preview.value)

        # symptoms
        diseases = results.get("diseases", [])
        if isinstance(diseases, list):
            symptoms_html = "".join(f"<div class='symptom-line'>• {d}</div>" for d in diseases)
        else:
            symptoms_html = f"<div class='symptom-line'>• {diseases}</div>"

        # advice split
        advice_items = [
            a.strip()
            for a in re.split(r"\d+\.\s*", str(results.get("advice", "")))
            if a.strip()
        ]
        advice_html = "".join(f"<div class='advice-line'>• {a}</div>" for a in advice_items) or "<div>• No advice available.</div>"

        res_html = f"""
        <div class="ai-report">
          <div class="ai-header">
            <span class="ai-header-icon">🧠</span>
            <div>
              <div class="ai-title">AI Diagnosis Report</div>
              <div class="ai-subtitle">Computer Vision • Plant Health</div>
            </div>
          </div>

          <div class="ai-pills">
            <div class="ai-pill plant">🌱 {results.get('name','Unknown')} ({results.get('id_conf','—')})</div>
            <div class="ai-pill health">❤️ {results.get('health','Health')}</div>
          </div>

          <div class="ai-section">
            <div class="ai-section-title">Detected Symptoms</div>
            <div class="ai-symptoms">{symptoms_html}</div>
          </div>

          <div class="ai-section">
            <div class="ai-section-title">Botanical Recommendation</div>
            <div class="ai-advice">{advice_html}</div>
          </div>
        </div>
        """

        with out_results:
            out_results.clear_output()
            display(widgets.HTML(res_html))

    except Exception as e:
        with out_results:
            out_results.clear_output()
            print(f"Analysis Error: {e}")

    finally:
        btn_analyze.disabled = False
        btn_analyze.description = "RUN ANALYSIS"

# Bind events
file_uploader.observe(on_upload_change, names="value")
btn_analyze.on_click(on_analyze_click)
btn_reset.on_click(on_reset_click)

# --- Render ---
final_ui = widgets.VBox(
    [style_html_upload, main_card_content, tips_card],
    layout=Layout(width="100%", align_items="center")
)
final_ui.add_class("fv-upload-scope")

if "wrap_screen" in globals():
    ui_screen = wrap_screen(final_ui)
else:
    ui_screen = final_ui

# If shell already exists, inject this screen
if "screens" in globals() and isinstance(screens, list) and len(screens) >= 2:
    # Upload screen should be index 1 in the final shell (overview=0, upload=1, index=2, rag=3)
    screens[1] = ui_screen
    if "screens_container" in globals():
        screens_container.children = tuple(screens)

display(ui_screen)


In [6]:
# =========================================================
# ONE CELL: PAPERS + UNIFORM_INDEX + Index Screen (FIXED UI)
# English UI only
# =========================================================

import re, json
from collections import defaultdict
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display

# -------------------------
# 0) PAPERS (5)
# -------------------------
PAPERS = [
    {
        "DocID": 1,
        "title": "Using Deep Learning for Image-Based Plant Disease Detection (Mohanty et al., 2016)",
        "url": "https://www.frontiersin.org/journals/plant-science/articles/10.3389/fpls.2016.01419/full",
        "text": """
Plant disease recognition from leaf images using CNNs. The model is trained/fine-tuned on labeled images across
multiple crops and diseases, learning cues such as lesions, spotting, discoloration, and texture. The work motivates
mobile/cloud decision support. It also highlights domain shift: field images include background clutter, lighting
variation, occlusion, and camera differences, requiring more robust datasets and methods for deployment.
"""
    },
    {
        "DocID": 2,
        "title": "Plant leaf disease classification using EfficientNet deep learning model (Atila et al., 2021)",
        "url": "https://www.sciencedirect.com/science/article/pii/S1574954120301321",
        "text": """
EfficientNet-based plant leaf disease classification. The pipeline uses preprocessing and augmentation
(rotation/flip/brightness) plus transfer learning. Evaluation uses accuracy, precision, recall and F1. EfficientNet can
reach strong accuracy with lower computation, supporting edge/mobile deployment. The paper emphasizes validation on
realistic field-like images for robustness.
"""
    },
    {
        "DocID": 3,
        "title": "Survey/Review: Deep learning for plant leaf disease detection and classification (review paper)",
        "url": "https://arxiv.org/pdf/2308.14087",
        "text": """
Survey of deep learning for plant disease detection/classification. It compares CNN backbones and transformer trends,
and discusses preprocessing, segmentation, augmentation, and evaluation with confusion matrices and precision/recall.
Main challenges: limited labeled data, class imbalance, and the gap between controlled datasets and in-the-wild images.
Suggested directions: domain adaptation, interpretability (heatmaps/attention maps), and diverse datasets for deployment.
"""
    },
    {
        "DocID": 4,
        "title": "Transformer/Attention-based Plant Disease Classification (Sensors/MDPI example)",
        "url": "https://www.mdpi.com/1424-8220/23/8/3955",
        "text": """
Transformer/attention-based plant disease classification. Self-attention captures global relationships across the leaf
and can focus on informative regions beyond local convolution patterns. The paper compares attention models to CNN
baselines using standard metrics and discusses robustness and data requirements. Attention maps support interpretability
for agricultural decision support.
"""
    },
    {
        "DocID": 5,
        "title": "Plant disease classification in the wild using vision transformers and mixture of experts (Salman et al., 2025)",
        "url": "https://www.frontiersin.org/journals/plant-science/articles/10.3389/fpls.2025.1522985/full",
        "text": """
In-the-wild plant disease classification under domain shift (backgrounds, lighting, occlusion, camera variability).
Proposes a vision-transformer approach with mixture-of-experts, where specialized experts handle different patterns and a
gating mechanism selects experts per image. The goal is improved generalization from curated datasets to field-like
images. Evaluation highlights robustness for precision agriculture deployment.
"""
    },
]

# -------------------------
# 1) STOPWORDS
# -------------------------
STOPWORDS = set("""
a an the and or but if then else for to of in on at by with from as is are was were be been being
this that these those it its into over under between among not no yes
we our you your they their he she his her them us
can could may might must should would
using use used method approach approaches model models
data results result paper study studies based
""".split())

# -------------------------
# 2) Preprocess
# -------------------------
def preprocess_text(text: str):
    text = (text or "").lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    return [t for t in text.split() if t and (t not in STOPWORDS) and (len(t) > 2)]

# -------------------------
# 3) Build UNIFORM_INDEX
# -------------------------
term_to_doclinks = defaultdict(list)

for p in PAPERS:
    did = p["DocID"]
    url = p["url"]
    for term in set(preprocess_text(p.get("text", ""))):
        term_to_doclinks[term].append({"DocID": did, "url": url})

UNIFORM_INDEX = []
for term, links in term_to_doclinks.items():
    links_sorted = sorted(links, key=lambda x: x["DocID"])
    numbered_links = [f'[{x["DocID"]}] {x["url"]}' for x in links_sorted]
    UNIFORM_INDEX.append({"term": term, "DocIDs": numbered_links})

UNIFORM_INDEX.sort(key=lambda x: x["term"])
TERM_TO_DOCIDS = {row["term"]: row["DocIDs"] for row in UNIFORM_INDEX}

with open("uniform_index.json", "w", encoding="utf-8") as f:
    json.dump(UNIFORM_INDEX, f, ensure_ascii=False, indent=2)

print("✅ Papers loaded:", len(PAPERS))
print("✅ UNIFORM_INDEX terms:", len(UNIFORM_INDEX))
print("✅ Saved: uniform_index.json")

# -------------------------
# 4) Search
# -------------------------
def search_using_index(query: str):
    q_terms = list(dict.fromkeys(preprocess_text(query)))
    if not q_terms:
        return []

    hits_per_doc = defaultdict(int)

    for term in set(q_terms):
        for entry in TERM_TO_DOCIDS.get(term, []):
            m = re.match(r"\[(\d+)\]\s+(.*)", entry)
            if not m:
                continue
            did = int(m.group(1))
            hits_per_doc[did] += 1

    ranked = sorted(hits_per_doc.items(), key=lambda x: x[1], reverse=True)

    results = []
    for did, _ in ranked:
        meta = next(p for p in PAPERS if p["DocID"] == did)
        results.append({"DocID": did, "title": meta["title"], "url": meta["url"]})
    return results

# -------------------------
# 5) Index Screen (FIXED sizing + Clear secondary)
# -------------------------
INDEX_FIX_CSS = widgets.HTML(r"""
<style>
/* consistent sizing inside index area */
.fv-index-scope .widget-text input{
  height:52px !important;
  line-height:52px !important;
  padding:0 16px !important;
  box-sizing:border-box !important;
  border-radius:14px !important;
}
.fv-index-scope .widget-dropdown select{
  height:52px !important;
  line-height:52px !important;
  padding:0 14px !important;
  box-sizing:border-box !important;
  border-radius:14px !important;
  background:#ffffff !important;
}
</style>
""")

def build_index_screen():
    q = widgets.Text(
        value="",
        placeholder="Type your search query...",
        layout=Layout(width="520px", height="52px")
    )

    topk = widgets.Dropdown(
        options=[("Top 1", 1), ("Top 2", 2), ("Top 3", 3), ("Top 4", 4)],
        value=3,
        layout=Layout(width="170px", height="52px")
    )

    btn_search = widgets.Button(description="Search", layout=Layout(height="52px"))
    btn_clear  = widgets.Button(description="Clear", layout=Layout(height="52px"))

    btn_search.add_class("fv-nav-btn")
    btn_clear.add_class("fv-nav-btn")
    btn_clear.add_class("is-clear")   # makes it white/secondary via Shell CSS

    results_box = widgets.VBox([], layout=Layout(width="100%"))

    def render_message(title, text):
        results_box.children = (widgets.HTML(f"""
        <div class="fv-card" style="padding:14px;">
          <div style="font-weight:900;color:#111827;font-size:16px;">{title}</div>
          <div style="color:#6b7280;font-size:14px;margin-top:6px;line-height:1.55;">{text}</div>
        </div>
        """),)

    def do_search(_=None):
        query = (q.value or "").strip()
        if not query:
            render_message("Ready", "Enter a term to search.")
            return

        results = search_using_index(query)
        if not results:
            render_message("No results", "Try different keywords (e.g., transformer, domain shift, augmentation).")
            return

        k = int(topk.value)
        items = []
        for i, r in enumerate(results[:k], start=1):
            items.append(widgets.HTML(f"""
            <div class="fv-card" style="padding:14px;">
              <div style="font-weight:900;color:#111827;font-size:15px;margin-bottom:6px;">
                {r['title']}
                <span style="margin-left:8px;background:#dcfce7;border:1px solid rgba(34,197,94,.25);
                             color:#166534;font-weight:900;border-radius:999px;padding:4px 10px;font-size:12px;">
                  Top {i}
                </span>
              </div>
              <div style="font-size:13px;">
                Source: <a href="{r['url']}" target="_blank" style="color:#2563eb;text-decoration:none;">open link</a>
              </div>
            </div>
            """))
        results_box.children = (widgets.VBox(items, layout=Layout(width="100%", gap="10px")),)

    def do_clear(_=None):
        q.value = ""
        render_message("Ready", "Enter a term to search.")

    btn_search.on_click(do_search)
    btn_clear.on_click(do_clear)
    q.on_submit(do_search)
    topk.observe(lambda ch: do_search(), names="value")

    header = widgets.HTML("""
    <div class="fv-card">
      <h3 style="margin:2px 0 10px 0;font-size:16px;color:#111827;font-weight:900;">
        Index Search
      </h3>
      <div style="color:#6b7280;font-size:14px;line-height:1.55;">
        Search the <b>UNIFORM_INDEX</b> built from 5 plant-disease detection papers.
      </div>
    </div>
    """)

    controls = widgets.HBox([q, topk, btn_search, btn_clear],
                            layout=Layout(width="100%", gap="10px"))
    controls.add_class("fv-index-scope")

    main = widgets.VBox(
        [INDEX_FIX_CSS, header, controls, widgets.HTML("<div style='height:10px'></div>"), results_box],
        layout=Layout(width="100%")
    )
    main.add_class("fv-index-scope")

    app = widgets.VBox([main], layout=Layout(width="100%"))
    app.add_class("fv-app")
    main.add_class("fv-main")

    do_clear()
    return app

def _wrap_if_needed(w):
    if "wrap_screen" in globals():
        return wrap_screen(w)
    return widgets.Box([w], layout=Layout(width="100%"))

index_screen = _wrap_if_needed(build_index_screen())

if "set_ai_screen" in globals():
    set_ai_screen(index_screen)
else:
    display(index_screen)


✅ Papers loaded: 5
✅ UNIFORM_INDEX terms: 136
✅ Saved: uniform_index.json


Box(children=(VBox(children=(VBox(children=(HTML(value='\n<style>\n/* consistent sizing inside index area */\n…

In [7]:
import ipywidgets as widgets
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import io, base64, json, requests
from datetime import datetime, timedelta

# =====================================================
# 1) Server Configuration
# =====================================================
BASE_URL = "https://server-cloud-v645.onrender.com"

def fetch_samples(limit=24):
    try:
        r = requests.get(f"{BASE_URL}/history", params={"feed": "json", "limit": limit}, timeout=90)
        r.raise_for_status()
        payload = r.json()
        rows = payload["data"] if isinstance(payload, dict) else payload
        samples = []
        for row in rows:
            raw = row.get("value", {})
            if isinstance(raw, str):
                raw = json.loads(raw)
            samples.append({
                "temperature": float(raw.get("temperature", 0)),
                "humidity": float(raw.get("humidity", 0)),
                "soil": float(raw.get("soil", 0)),
                "timestamp": row.get("created_at")
            })
        return samples[::-1]  # oldest -> newest
    except Exception as e:
        print(f"Error fetching from server: {e}")
        return []

def format_server_time(ts):
    if not ts:
        return "—"
    dt = datetime.fromisoformat(ts.replace("Z", "+00:00"))
    return dt.astimezone().strftime("%d/%m/%Y %H:%M:%S")

# =====================================================
# 2) ALERTS FEATURE (NEW FUNCTIONAL FEATURE)
# =====================================================
ALERT_HISTORY = []          # keeps last alerts
ALERT_LAST_SEEN = {}        # suppress spam
MAX_ALERTS = 25             # max history size
SUPPRESS_MINUTES = 8        # don't repeat same alert too often

# thresholds (you can tweak)
TH = {
    "SOIL_CRIT": 25,
    "SOIL_WARN": 35,
    "TEMP_WARN_HI": 32,
    "TEMP_CRIT_HI": 35,
    "TEMP_WARN_LO": 10,
    "TEMP_CRIT_LO": 6,
    "HUM_WARN_LO": 35,
    "HUM_WARN_HI": 85,
    "SOIL_DROP_TREND": 12,  # % drop in short time
}

def _now():
    return datetime.now()

def _mk_alert(code, level, title, message, time_label):
    return {
        "code": code,
        "level": level,          # INFO / WARNING / CRITICAL
        "title": title,
        "message": message,
        "time": time_label,
        "created_at": _now().isoformat(timespec="seconds")
    }
def save_alert_to_firebase(alert: dict):
    """
    Save a single alert to Firebase under /alerts/history
    Key = timestamp (ISO) to keep ordering
    """
    try:
        key = alert["created_at"].replace(":", "-")
        url = f"{FIREBASE_DB_URL}/alerts/history/{key}.json"
        r = requests.put(url, json=alert, timeout=10)
        r.raise_for_status()
    except Exception as e:
        print("Firebase alert save failed:", e)

def detect_alerts(samples):
    """Return list of alerts based on latest sample + short trend."""
    if not samples:
        return []

    latest = samples[-1]
    t = latest["temperature"]
    h = latest["humidity"]
    s = latest["soil"]
    time_label = format_server_time(latest.get("timestamp"))

    alerts = []

    # --- Soil Moisture ---
    if s < TH["SOIL_CRIT"]:
        alerts.append(_mk_alert(
            "SOIL_LOW", "CRITICAL",
            "Low Soil Moisture",
            f"Soil moisture is very low ({s:.0f}%). Irrigation is required.",
            time_label
        ))
    elif s < TH["SOIL_WARN"]:
        alerts.append(_mk_alert(
            "SOIL_LOW", "WARNING",
            "Soil Moisture Dropping",
            f"Soil moisture is below recommended ({s:.0f}%). Consider watering soon.",
            time_label
        ))

    # --- Temperature ---
    if t >= TH["TEMP_CRIT_HI"]:
        alerts.append(_mk_alert(
            "TEMP_HIGH", "CRITICAL",
            "High Temperature",
            f"Temperature is too high ({t:.1f}°C). Risk of heat stress.",
            time_label
        ))
    elif t >= TH["TEMP_WARN_HI"]:
        alerts.append(_mk_alert(
            "TEMP_HIGH", "WARNING",
            "Temperature Above Normal",
            f"Temperature is elevated ({t:.1f}°C). Monitor ventilation/shading.",
            time_label
        ))

    if t <= TH["TEMP_CRIT_LO"]:
        alerts.append(_mk_alert(
            "TEMP_LOW", "CRITICAL",
            "Low Temperature",
            f"Temperature is too low ({t:.1f}°C). Risk of cold damage.",
            time_label
        ))
    elif t <= TH["TEMP_WARN_LO"]:
        alerts.append(_mk_alert(
            "TEMP_LOW", "WARNING",
            "Temperature Below Normal",
            f"Temperature is low ({t:.1f}°C). Consider protection at night.",
            time_label
        ))

    # --- Humidity ---
    if h <= TH["HUM_WARN_LO"]:
        alerts.append(_mk_alert(
            "HUM_LOW", "WARNING",
            "Low Humidity",
            f"Humidity is low ({h:.0f}%). Plants may dry faster than usual.",
            time_label
        ))
    elif h >= TH["HUM_WARN_HI"]:
        alerts.append(_mk_alert(
            "HUM_HIGH", "WARNING",
            "High Humidity",
            f"Humidity is high ({h:.0f}%). Risk of fungal disease increases.",
            time_label
        ))

    # --- Trend alert (interesting extra feature) ---
    # soil drop quickly over last ~4 samples
    if len(samples) >= 4:
        s_old = samples[-4]["soil"]
        drop = s_old - s
        if drop >= TH["SOIL_DROP_TREND"]:
            alerts.append(_mk_alert(
                "SOIL_TREND", "WARNING",
                "Fast Soil Moisture Drop",
                f"Soil dropped quickly by {drop:.0f}% (from {s_old:.0f}% to {s:.0f}%). Check leaks or irrigation schedule.",
                time_label
            ))

    return alerts

def push_alerts(alerts):
    global ALERT_HISTORY, ALERT_LAST_SEEN

    if not alerts:
        return

    now = _now()
    for a in alerts:
        key = f"{a['code']}|{a['level']}"
        last = ALERT_LAST_SEEN.get(key)
        if last and (now - last) < timedelta(minutes=SUPPRESS_MINUTES):
            continue

        ALERT_LAST_SEEN[key] = now
        ALERT_HISTORY.insert(0, a)

        # ✅ SAVE TO FIREBASE
        save_alert_to_firebase(a)

    ALERT_HISTORY = ALERT_HISTORY[:MAX_ALERTS]
def load_alerts_from_firebase(limit=50):
    try:
        url = f"{FIREBASE_DB_URL}/alerts/history.json"
        params = {
            "orderBy": '"$key"',
            "limitToLast": limit
        }
        r = requests.get(url, params=params, timeout=10)
        r.raise_for_status()
        data = r.json()

        if not data:
            return []

        alerts = list(data.values())
        alerts.sort(key=lambda a: a.get("created_at", ""), reverse=True)
        return alerts

    except Exception as e:
        print("Failed to load alerts:", e)
        return []


def clear_alerts(_=None):
    global ALERT_HISTORY, ALERT_LAST_SEEN
    ALERT_HISTORY = []
    ALERT_LAST_SEEN = {}
    alerts_panel.value = render_alerts_html([])

def render_alerts_html(alerts):
    # counts
    c_crit = sum(1 for a in alerts if a["level"] == "CRITICAL")
    c_warn = sum(1 for a in alerts if a["level"] == "WARNING")
    c_info = sum(1 for a in alerts if a["level"] == "INFO")

    def badge(level):
        if level == "CRITICAL":
            return "<span class='al-badge al-crit'>CRITICAL</span>"
        if level == "WARNING":
            return "<span class='al-badge al-warn'>WARNING</span>"
        return "<span class='al-badge al-info'>INFO</span>"

    if not alerts:
        items = "<div class='al-empty'>No alerts yet. System looks stable ✅</div>"
    else:
        rows = []
        for a in alerts[:12]:
            rows.append(f"""
              <div class="al-row">
                <div class="al-left">
                  {badge(a["level"])}
                  <div class="al-text">
                    <div class="al-title">{a["title"]}</div>
                    <div class="al-msg">{a["message"]}</div>
                  </div>
                </div>
                <div class="al-time">{a["time"]}</div>
              </div>
            """)
        items = "\n".join(rows)

    return f"""
    <div class="alerts-card">
      <div class="alerts-head">
        <div>
          <div class="alerts-h1">Alerts Center</div>
          <div class="alerts-h2">Automatic warnings based on sensor thresholds + trends</div>
        </div>
        <div class="alerts-counts">
          <span class="al-pill al-crit">🔴 {c_crit}</span>
          <span class="al-pill al-warn">🟠 {c_warn}</span>
          <span class="al-pill al-info">🔵 {c_info}</span>
        </div>
      </div>
      <div class="alerts-body">
        {items}
      </div>
    </div>
    """

# =====================================================
# 3) UI Styling (Dashboard + Alerts)
# =====================================================
style_css = """
<style>
  @import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700;800&display=swap');

  .dashboard-scope { font-family: 'Inter', sans-serif; font-size: 16px; }

  .dashboard-scope .widget-vbox,
  .dashboard-scope .widget-hbox,
  .dashboard-scope .widget-box {
    background-color: transparent !important;
  }

  .slider-wrapper-card {
    background: #ffffff !important;
    border: 1px solid #e2e8f0 !important;
    border-radius: 20px !important;
    padding: 20px !important;
    box-shadow: 0 4px 15px rgba(0, 0, 0, 0.04) !important;
    margin-bottom: 25px !important;
  }

  .widget-readout { color: #0f172a !important; font-weight: 700 !important; }
  .slider-label { font-weight: 700; font-size: 15px; color: #0f172a; margin-bottom: 12px; }

  .sensor-card { border-radius: 24px; padding: 24px; box-shadow: 0 4px 12px rgba(0, 0, 0, 0.04); color: #1e293b; transition: transform 0.2s ease; }
  .sensor-card:hover { transform: translateY(-3px); }
  .sensor-card-orange { background: linear-gradient(135deg, #fff0e0 0%, #ffe0c0 100%); }
  .sensor-card-blue { background: linear-gradient(135deg, #e0f0ff 0%, #c0e0ff 100%); }
  .sensor-card-green { background: linear-gradient(135deg, #e0ffe0 0%, #c0ffc0 100%); }

  .metric-title { font-size: 15px; font-weight: 600; color: #475569; margin-bottom: 8px; }
  .metric-value { font-size: 36px; font-weight: 800; letter-spacing: -1px; margin-bottom: 4px; }
  .metric-unit { font-size: 20px; font-weight: 600; opacity: 0.7; margin-left: 4px; }
  .metric-status { font-size: 13px; font-weight: 600; color: #64748b; }

  .ui-slider-horizontal { height: 6px !important; background: #e2e8f0 !important; border: none !important; border-radius: 3px !important; }
  .ui-slider-handle {
    width: 18px !important; height: 18px !important; border-radius: 50% !important;
    background: #ffffff !important; border: 3px solid #94a3b8 !important; top: -6px !important;
    outline: none !important; box-shadow: 0 2px 4px rgba(0,0,0,0.1) !important;
  }

  /* ===== Alerts Center (NEW) ===== */
  .alerts-card{
    background:#ffffff;
    border:1px solid #e2e8f0;
    border-radius:22px;
    box-shadow:0 10px 28px rgba(15,23,42,.06);
    padding:18px 18px 10px 18px;
  }
  .alerts-head{
    display:flex;
    justify-content:space-between;
    gap:12px;
    flex-wrap:wrap;
    align-items:flex-end;
    padding-bottom:12px;
    border-bottom:1px solid rgba(15,23,42,.06);
  }
  .alerts-h1{ font-size:16px; font-weight:900; color:#0f172a; }
  .alerts-h2{ font-size:12px; color:#64748b; margin-top:4px; }

  .alerts-counts{ display:flex; gap:10px; }
  .al-pill{
    display:inline-flex; align-items:center; gap:6px;
    padding:6px 10px; border-radius:999px;
    font-size:12px; font-weight:900;
    border:1px solid rgba(15,23,42,.08);
    background:#f8fafc;
    color:#0f172a;
  }
  .al-pill.al-crit{ background:#fee2e2; border-color:rgba(239,68,68,.25); color:#991b1b; }
  .al-pill.al-warn{ background:#ffedd5; border-color:rgba(249,115,22,.25); color:#9a3412; }
  .al-pill.al-info{ background:#dbeafe; border-color:rgba(59,130,246,.25); color:#1e40af; }

  .alerts-body{ padding-top:12px; }

  .al-row{
    display:flex;
    justify-content:space-between;
    gap:12px;
    padding:10px 10px;
    border-radius:14px;
    border:1px solid rgba(15,23,42,.06);
    background:#ffffff;
    margin-bottom:10px;
  }
  .al-left{ display:flex; gap:10px; align-items:flex-start; }
  .al-text{ display:flex; flex-direction:column; }
  .al-title{ font-size:13px; font-weight:900; color:#0f172a; }
  .al-msg{ font-size:12px; color:#475569; margin-top:2px; line-height:1.5; }
  .al-time{ font-size:12px; color:#94a3b8; white-space:nowrap; }

  .al-badge{
    font-size:11px; font-weight:900;
    padding:6px 10px; border-radius:999px;
    border:1px solid rgba(15,23,42,.08);
    background:#f8fafc; color:#0f172a;
    min-width:86px; text-align:center;
  }
  .al-badge.al-crit{ background:#fee2e2; border-color:rgba(239,68,68,.25); color:#991b1b; }
  .al-badge.al-warn{ background:#ffedd5; border-color:rgba(249,115,22,.25); color:#9a3412; }
  .al-badge.al-info{ background:#dbeafe; border-color:rgba(59,130,246,.25); color:#1e40af; }

  .al-empty{
    padding:12px;
    border:1px dashed rgba(15,23,42,.14);
    border-radius:14px;
    background:#f8fafc;
    color:#64748b;
    font-size:13px;
  }
</style>
"""

# =====================================================
# 4) UI Helpers
# =====================================================
def create_metric_card_new(title, value, unit, status, color_theme, icon_svg):
    return widgets.HTML(f"""
        <div class="sensor-card sensor-card-{color_theme}">
            <div style="display:flex; justify-content:space-between; align-items:flex-start;">
                <div>
                    <div class="metric-title">{title}</div>
                    <div class="metric-value">{value}<span class="metric-unit">{unit}</span></div>
                    <div class="metric-status">{status}</div>
                </div>
                <div style="opacity:0.6; transform:scale(1.1);">{icon_svg}</div>
            </div>
        </div>
    """, layout=widgets.Layout(width="33%"))

def create_chart(x_labels, y_data, title, line_color):
    plt.ioff()
    fig, ax = plt.subplots(figsize=(6, 3), dpi=100)
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')
    ax.plot(y_data, color=line_color, linewidth=2.5, marker='o', markersize=4,
            markerfacecolor='white', markeredgewidth=1.5)
    ax.fill_between(range(len(y_data)), y_data, alpha=0.2, color=line_color)
    ax.set_title(title, loc="left", color="#334155", fontsize=11, fontweight='700', pad=15)
    step = max(1, len(x_labels)//5)
    ax.set_xticks(range(0, len(x_labels), step))
    ax.set_xticklabels([x_labels[i] for i in range(0, len(x_labels), step)], color='#0f172a', fontsize=8)
    ax.tick_params(axis='y', colors='#0f172a', labelsize=8)
    for spine in ax.spines.values():
        spine.set_visible(False)
    ax.spines['bottom'].set_visible(True)
    ax.spines['bottom'].set_color('#e2e8f0')
    ax.grid(axis='y', color='#f1f5f9', linestyle='-')

    buf = io.BytesIO()
    plt.tight_layout()
    plt.savefig(buf, format="png", transparent=False)
    plt.close(fig)

    img = base64.b64encode(buf.getvalue()).decode()
    return widgets.HTML(
        f"""<div style="background:#ffffff;border-radius:20px;padding:15px;border:1px solid #e2e8f0;
                     box-shadow:0 4px 15px rgba(0,0,0,0.04);">
               <img src='data:image/png;base64,{img}' style='width:100%; border-radius:12px;'>
             </div>""",
        layout=widgets.Layout(width="50%")
    )

# =====================================================
# 5) Build Dashboard UI
# =====================================================
icon_temp = '<svg width="24" height="24" fill="none" stroke="#f97316" stroke-width="2"><path d="M14 14.76V3.5a2.5 2.5 0 0 0-5 0v11.26a4.5 4.5 0 1 0 5 0z"/></svg>'
icon_hum  = '<svg width="24" height="24" fill="none" stroke="#3b82f6" stroke-width="2"><path d="M12 2.69l5.66 5.66a8 8 0 1 1-11.31 0z"/></svg>'
icon_soil = '<svg width="24" height="24" fill="none" stroke="#10b981" stroke-width="2"><path d="M12 22s8-4 8-10V5l-8-3-8 3v7c0 6 8 10 8 10z"/></svg>'

output = widgets.VBox()
hours_slider = widgets.IntSlider(value=24, min=1, max=72, continuous_update=False,
                                 layout=widgets.Layout(width="100%"))
slider_label = widgets.HTML("<div class='slider-label'>History Range (Hours)</div>")
slider_container = widgets.VBox([slider_label, hours_slider])
slider_container.add_class("slider-wrapper-card")

# Alerts panel widgets (persistent)
alerts_panel = widgets.HTML(render_alerts_html([]))
btn_ack = widgets.Button(description="Acknowledge All", layout=widgets.Layout(width="200px", height="44px"))
btn_ack.on_click(clear_alerts)

ack_row = widgets.HBox([btn_ack], layout=widgets.Layout(justify_content="flex-end", margin="0 0 10px 0"))

def refresh(_=None):
    output.children = [widgets.HTML("<div style='padding:50px; text-align:center; color:#94a3af;'>Syncing with sensors...</div>")]
    samples = fetch_samples(hours_slider.value)
    if not samples:
        return

    latest = samples[-1]

    # ===== NEW: Alerts detection + history =====
    new_alerts = detect_alerts(samples)
    push_alerts(new_alerts)
    alerts_panel.value = render_alerts_html(ALERT_HISTORY)

    # metric cards
    cards = widgets.HBox([
        create_metric_card_new("Temperature", f"{latest['temperature']:.1f}", "°C", "Live", "orange", icon_temp),
        create_metric_card_new("Humidity", f"{latest['humidity']:.0f}", "%", "Live", "blue", icon_hum),
        create_metric_card_new("Soil Moisture", f"{latest['soil']:.0f}", "%", "Live", "green", icon_soil),
    ], layout=widgets.Layout(gap="24px"))

    times = [format_server_time(s["timestamp"])[11:] for s in samples]
    chart1 = create_chart(times, [s["temperature"] for s in samples], "Climate Trend (°C)", "#f97316")
    chart2 = create_chart(times, [s["soil"] for s in samples], "Soil Moisture History (%)", "#10b981")
    charts = widgets.HBox([chart1, chart2], layout=widgets.Layout(gap="24px", margin="30px 0"))

    # diagnostics line (still useful)
    status_color = "#ef4444" if latest['soil'] < TH["SOIL_WARN"] else "#10b981"
    status_text = "Action Required: Soil Moisture Low" if latest['soil'] < TH["SOIL_WARN"] else "All Systems Nominal"

    diagnostics = widgets.HTML(f"""
    <div style="background:white;border:1px solid #e2e8f0;border-radius:16px;padding:20px;
                display:flex;align-items:center;box-shadow:0 4px 15px rgba(0,0,0,0.04);">
      <div style="width:12px;height:12px;background:{status_color};border-radius:50%;margin-right:15px;
                  box-shadow:0 0 8px {status_color}40;"></div>
      <div>
        <div style="color:#0f172a;font-weight:800;font-size:15px;">System Diagnostics: {status_text}</div>
        <div style="color:#64748b;font-size:12px;margin-top:4px;">Last sync: {format_server_time(latest['timestamp'])}</div>
      </div>
    </div>
    """)
    output.children = [cards, charts, diagnostics]

hours_slider.observe(refresh, "value")
refresh()

dashboard = widgets.VBox([
    widgets.HTML(style_css),
    widgets.VBox([
        slider_container,
        output
    ], layout=widgets.Layout(padding="35px", background_color="#ffffff", border_radius="32px",
                             box_shadow="0 10px 30px rgba(0,0,0,0.05)"))
])
dashboard.add_class("dashboard-scope")

display(dashboard)


In [8]:
# =====================================================
# ALERTS SCREEN (FULL HISTORY)
# =====================================================

ALERT_HISTORY = load_alerts_from_firebase()
alerts_panel_full = widgets.HTML(render_alerts_html(ALERT_HISTORY))


btn_ack_all = widgets.Button(
    description="Acknowledge All",
    layout=widgets.Layout(width="220px", height="44px")
)
btn_ack_all.on_click(clear_alerts)

def refresh_alerts_screen():
    alerts_panel_full.value = render_alerts_html(ALERT_HISTORY)

alerts_screen = widgets.VBox(
    [
        widgets.HTML("<h2 style='margin-bottom:4px'>Alerts Center</h2>"),
        widgets.HTML("<div style='color:#64748b;margin-bottom:16px'>All system alerts (thresholds + trends)</div>"),
        widgets.HBox([btn_ack_all], layout=widgets.Layout(justify_content="flex-end")),
        alerts_panel_full
    ],
    layout=widgets.Layout(gap="16px", padding="24px")
)


In [9]:
# =========================================================
# RAG SCREEN (Auto AI if available) — FIXED THEME
# Runs BEFORE Shell
# Exposes: rag_screen
# =========================================================

import re
import ipywidgets as widgets
from ipywidgets import Layout

# -------------------------
# Safety checks (Index only)
# -------------------------
need = ["PAPERS", "UNIFORM_INDEX", "search_using_index"]
missing = [x for x in need if x not in globals()]
if missing:
    raise RuntimeError("RAG requires Index first. Missing: " + ", ".join(missing))

# AI is optional (from Block 1)
_HAS_RAG_AI = ("rag_ai_answer" in globals())

# -------------------------
# CSS (SCOPED)
# -------------------------
RAG_CSS = widgets.HTML(r"""
<style>
.fv-classic-rag{
  font-family: ui-sans-serif, system-ui, -apple-system, "Segoe UI", Roboto, Arial;
  font-size: 16px;
}

/* page */
.fv-classic-rag .rag-page{
  background:#ffffff;
  border:1px solid rgba(17,24,39,.10);
  border-radius:16px;
  box-shadow:0 12px 26px rgba(17,24,39,.06);
}

/* header */
.fv-classic-rag .rag-header{
  padding:16px 18px;
  border-bottom:1px solid rgba(17,24,39,.08);
  display:flex;
  justify-content:space-between;
  gap:14px;
  flex-wrap:wrap;
  align-items:center;
}
.fv-classic-rag .rag-h1{ font-size:18px; font-weight:900; color:#111827; margin:0; }
.fv-classic-rag .rag-h2{ font-size:13px; color:#6b7280; margin-top:6px; line-height:1.5; }

.fv-classic-rag .rag-body{ padding:16px 18px 18px }
.fv-classic-rag .rag-label{ font-size:13px; font-weight:900; margin-bottom:6px; color:#111827; }

/* input */
.fv-classic-rag .widget-text input{
  background:#ffffff !important;
  color:#0f172a !important;
  border:1px solid #e2e8f0 !important;
  border-radius:14px !important;
  height:52px !important;
  line-height:52px !important;
  padding:0 16px !important;
  font-size:16px !important;
  box-shadow:0 10px 26px rgba(15,23,42,0.06) !important;
}
.fv-classic-rag .widget-text input::placeholder{ color:#94a3b8 !important; }
.fv-classic-rag .widget-text input:focus{
  outline:none !important;
  border-color:rgba(34,197,94,.6) !important;
  box-shadow:0 0 0 3px rgba(34,197,94,.18), 0 10px 26px rgba(15,23,42,0.06) !important;
}

/* buttons */
.fv-classic-rag .fv-nav-btn,
.fv-classic-rag .fv-nav-btn button{
  background: linear-gradient(180deg,#22c55e 0%, #16a34a 100%) !important;
  color:#ffffff !important;
  border:0 !important;
  border-radius:14px !important;
  height:52px !important;
  padding:0 22px !important;
  font-weight:900 !important;
  font-size:15px !important;
  box-shadow:0 14px 28px rgba(34,197,94,.28) !important;
  cursor:pointer !important;
}
.fv-classic-rag .fv-nav-btn:hover,
.fv-classic-rag .fv-nav-btn button:hover{
  filter:brightness(.98) !important;
  transform:translateY(-1px) !important;
}
.fv-classic-rag .fv-nav-btn:disabled,
.fv-classic-rag .fv-nav-btn button:disabled{
  opacity:0.65 !important;
  cursor:not-allowed !important;
  transform:none !important;
}

/* clear secondary */
.fv-classic-rag .is-clear,
.fv-classic-rag .is-clear button{
  background:#ffffff !important;
  color:#334155 !important;
  border:1px solid #e2e8f0 !important;
  box-shadow:0 10px 22px rgba(15,23,42,.06) !important;
}

/* divider + answer */
.fv-classic-rag .rag-divider{ height:1px; background:rgba(17,24,39,.08); margin:14px 0; }

.fv-classic-rag .rag-answer{
  background:#fbfbfb;
  border:1px solid rgba(17,24,39,.10);
  border-radius:12px;
  padding:12px;
  font-size:14px;
  line-height:1.7;
  white-space:pre-wrap;
  color:#0f172a;
}

.fv-classic-rag .rag-answer.ai{
  background: linear-gradient(180deg,#ecfdf5 0%, #f0fdf4 100%);
  border-left:6px solid #22c55e;
}

.fv-classic-rag .rag-sources{ font-size:13px; color:#6b7280; line-height:1.7; }

.fv-classic-rag .rag-msg{
  border:1px dashed rgba(17,24,39,.18);
  border-radius:12px;
  padding:12px;
  font-size:14px;
  color:#6b7280;
}

.fv-classic-rag .rag-chip{
  display:inline-flex;
  align-items:center;
  gap:8px;
  padding:8px 12px;
  border-radius:999px;
  font-size:12px;
  font-weight:900;
  border:1px solid rgba(34,197,94,.20);
  background:#dcfce7;
  color:#166534;
}
.fv-classic-rag .rag-chip.off{
  border:1px solid rgba(100,116,139,.20);
  background:#f1f5f9;
  color:#334155;
}
</style>
""")

# -------------------------
# helpers (classic retrieval summary)
# -------------------------
def _split_sentences(text):
    text = (text or "").replace("\n", " ").strip()
    return [s.strip() for s in re.split(r"(?<=[.!?])\s+", text) if s.strip()]

def _pick_sentences(text, terms, k=2):
    sents = _split_sentences(text)
    picked = [s for s in sents if any(t in s.lower() for t in terms)]
    return (picked or sents)[:k]

def rag_answer_classic(query, retrieved, max_docs=3):
    if not retrieved:
        return "No relevant information found.", []

    terms = query.lower().split()
    answers, sources = [], []

    for r in retrieved[:max_docs]:
        meta = next(p for p in PAPERS if p["DocID"] == r["DocID"])
        sents = _pick_sentences(meta["text"], terms, k=2)
        if sents:
            answers.append(" ".join(sents))
        sources.append(meta)

    return "\n\n".join(answers), sources

def _build_context(srcs, max_chars=1400):
    chunks = []
    for s in srcs[:3]:
        t = " ".join(_split_sentences(s["text"])[:3])
        chunks.append(f"- {s['title']}: {t}")
    ctx = "\n".join(chunks)
    return ctx[:max_chars]

# -------------------------
# Build screen
# -------------------------
def build_rag_screen():
    q = widgets.Text(
        placeholder="Type your question here...",
        layout=Layout(width="100%", height="52px")
    )

    btn_answer = widgets.Button(description="Answer", layout=Layout(width="140px", height="52px"))
    btn_clear  = widgets.Button(description="Clear",  layout=Layout(width="140px", height="52px"))
    btn_answer.add_class("fv-nav-btn")
    btn_clear.add_class("fv-nav-btn")
    btn_clear.add_class("is-clear")

    chip = widgets.HTML(
        "<div class='rag-chip'>🧠 AI Enabled</div>" if _HAS_RAG_AI
        else "<div class='rag-chip off'>🧠 AI not loaded (run Block 1)</div>"
    )

    answer = widgets.HTML()
    sources = widgets.HTML()

    def ready():
        answer.value = """
        <div class="rag-msg">
          Ask a question about plant disease detection.<br>
          Example: <i>What is domain shift?</i>
        </div>
        """
        sources.value = ""

    def ask(_=None):
        query = q.value.strip()
        if not query:
            ready()
            return

        btn_answer.disabled = True
        btn_answer.description = "WORKING..."

        try:
            docs = search_using_index(query)

            classic_text, srcs = rag_answer_classic(query, docs)

            # sources list
            sources.value = "<br>".join(
                f'• <a href="{s["url"]}" target="_blank">{s["title"]}</a>'
                for s in srcs
            )

            # AUTO: AI if available, else classic
            if _HAS_RAG_AI and srcs:
                ctx = _build_context(srcs)
                ai = rag_ai_answer(query, ctx)  # from Block 1
                answer.value = f'<div class="rag-answer ai">{ai}</div>'
            else:
                answer.value = f'<div class="rag-answer">{classic_text}</div>'

        except Exception as e:
            answer.value = f'<div class="rag-answer">Error: {e}</div>'
            sources.value = ""
        finally:
            btn_answer.disabled = False
            btn_answer.description = "Answer"

    def clear(_=None):
        q.value = ""
        ready()

    btn_answer.on_click(ask)
    btn_clear.on_click(clear)
    q.on_submit(lambda _: ask())

    header = widgets.HTML("""
    <div>
      <div class="rag-h1">RAG</div>
      <div class="rag-h2">Index retrieval + AI answer (auto if available)</div>
    </div>
    """)

    header_row = widgets.HBox(
        [widgets.VBox([header]), widgets.VBox([chip], layout=Layout(align_items="flex-end"))],
        layout=Layout(width="100%", justify_content="space-between", align_items="center")
    )

    top_controls = widgets.HBox(
        [q, btn_answer, btn_clear],
        layout=Layout(width="100%", gap="12px", align_items="center")
    )

    body = widgets.VBox([
        widgets.HTML('<div class="rag-label">Question</div>'),
        top_controls,
        widgets.HTML('<div class="rag-divider"></div>'),
        widgets.HTML('<div class="rag-label">Answer</div>'),
        answer,
        widgets.HTML('<div class="rag-divider"></div>'),
        widgets.HTML('<div class="rag-label">Sources</div>'),
        sources
    ], layout=Layout(width="100%"))

    page = widgets.VBox(
        [widgets.Box([header_row], layout=Layout(width="100%")), widgets.VBox([body], layout=Layout(padding="16px"))],
        layout=Layout(width="100%")
    )
    page.add_class("rag-page")

    root = widgets.VBox([RAG_CSS, page], layout=Layout(width="100%"))
    root.add_class("fv-classic-rag")
    root.add_class("fv-rag-scope")

    ready()
    return root

rag_screen = build_rag_screen()


In [10]:
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display

# =========================
# ALERTS SCREEN
# =========================

def build_alerts_screen():
    title = widgets.HTML("""
    <div style="font-size:22px;font-weight:900;color:#0f172a;margin-bottom:6px;">
      Alerts Center
    </div>
    <div style="color:#64748b;font-size:14px;margin-bottom:18px;">
      Full alert history generated from sensors and trends
    </div>
    """)

    alerts_box = widgets.HTML()

    def refresh_alerts():
        alerts_box.value = render_alerts_html(ALERT_HISTORY)

    btn_refresh = widgets.Button(
        description="Refresh",
        layout=Layout(width="140px", height="44px")
    )
    btn_refresh.on_click(lambda _: refresh_alerts())

    btn_clear = widgets.Button(
        description="Clear All Alerts",
        layout=Layout(width="180px", height="44px")
    )
    btn_clear.add_class("is-clear")
    btn_clear.on_click(clear_alerts)

    controls = widgets.HBox(
        [btn_refresh, btn_clear],
        layout=Layout(gap="12px", margin="0 0 16px 0")
    )

    refresh_alerts()

    page = widgets.VBox(
        [title, controls, alerts_box],
        layout=Layout(
            width="100%",
            padding="24px",
            background_color="#ffffff",
            border_radius="24px",
            box_shadow="0 10px 30px rgba(0,0,0,0.06)"
        )
    )

    return page

alerts_screen = build_alerts_screen()


In [11]:
import ipywidgets as widgets
from IPython.display import display, HTML

# =================================================
# 1) CSS — Scoped Presenter (NO global body styles)
# =================================================
display(HTML(r"""
<style>
/* ================================
   FloraVision Shell (SCOPED)
   ================================ */
.fv-shell{
  width:100%;
  display:flex;
  flex-direction:column;
  background:#f1f5f9;
  min-height:100vh;
  box-sizing:border-box;
  padding-bottom:40px;
  font-family:'Segoe UI', Roboto, Helvetica, Arial, sans-serif;
  font-size:16px;
}

/* ================================
   Header
   ================================ */
.fv-shell .fv-shell-header{
  width:100%;
  background:#ffffff;
  border-bottom:1px solid #e2e8f0;
  box-shadow:0 1px 3px rgba(0,0,0,0.05);
  display:flex;
  flex-direction:column;
}

.fv-shell .fv-shell-header-top{
  display:flex;
  justify-content:space-between;
  align-items:center;
  padding:12px 40px;
  border-bottom:1px solid #f1f5f9;
  gap:16px;
  flex-wrap:wrap;
}

.fv-shell .fv-shell-logo-area{ display:flex; align-items:center; gap:12px; }
.fv-shell .fv-shell-logo-box{
  width:36px; height:36px;
  background:#22c55e;
  border-radius:10px;
  display:flex; align-items:center; justify-content:center;
  color:#fff;
}

.fv-shell .fv-shell-title{ font-size:24px; font-weight:900; color:#0f172a; }
.fv-shell .fv-shell-subtitle{ font-size:14px; color:#16a34a; font-weight:700; }

.fv-shell .fv-shell-user-area{ display:flex; align-items:center; gap:12px; flex-wrap:wrap; }
.fv-shell .fv-shell-points{
  background:#fffbeb;
  color:#b45309;
  border:1px solid #fcd34d;
  font-size:13px;
  font-weight:900;
  padding:6px 14px;
  border-radius:10px;
}
.fv-shell .fv-shell-welcome{ font-size:14px; color:#16a34a; font-weight:800; }

/* ================================
   Tabs (Fix clipping with bigger fonts)
   ================================ */
.fv-shell .fv-shell-tabs{
  display:flex;
  align-items:center;
  padding:0 40px;
  min-height:60px;
  gap:8px;
  background:#ffffff;
  flex-wrap:wrap;
}

.fv-shell .fv-shell-tabbtn{
  background:transparent !important;
  border:none !important;

  height:52px !important;
  padding:0 22px !important;

  font-size:16px !important;
  font-weight:800 !important;
  color:#64748b !important;

  cursor:pointer !important;
  border-bottom:2px solid transparent !important;
  transition:all .2s !important;

  white-space:nowrap !important;
  overflow:visible !important;
  text-overflow:clip !important;
  flex:0 0 auto !important;
  min-width:160px !important;

  outline:none !important;
  box-shadow:none !important;
}

.fv-shell .fv-shell-tabbtn:hover{
  color:#0f172a !important;
  background:#f8fafc !important;
  border-radius:12px !important;
}

.fv-shell .fv-shell-tabbtn.fv-active{
  color:#15803d !important;
  background:#f0fdf4 !important;
  border-bottom:3px solid #16a34a !important;
  border-radius:12px !important;
}

/* ================================
   Content
   ================================ */
.fv-shell .fv-shell-content{
  padding:24px 40px;
  width:100%;
  box-sizing:border-box;
  background:transparent;
}

.fv-shell .fv-shell-screen{
  width:100%;
  min-height:72vh;
}

/* ================================
   Index / RAG Controls (SCOPED)
   ================================ */
.fv-shell .fv-index-scope .widget-label,
.fv-shell .fv-index-scope label{
  color:#0f172a !important;
  font-weight:900 !important;
  font-size:14px !important;
}

.fv-shell .fv-index-scope .widget-text input,
.fv-shell .fv-index-scope input[type="text"]{
  background:#ffffff !important;
  color:#0f172a !important;
  border:1px solid #e2e8f0 !important;
  border-radius:14px !important;
  height:52px !important;
  line-height:52px !important;
  padding:0 16px !important;
  font-size:16px !important;
  box-shadow:0 10px 26px rgba(15,23,42,0.06) !important;
}

.fv-shell .fv-index-scope .widget-text input::placeholder{ color:#94a3b8 !important; }

.fv-shell .fv-index-scope .widget-dropdown select{
  background:#ffffff !important;
  color:#0f172a !important;
  border:1px solid #e2e8f0 !important;
  border-radius:14px !important;
  height:52px !important;
  padding:0 14px !important;
  font-size:15px !important;
  box-shadow:0 10px 26px rgba(15,23,42,0.06) !important;
}

.fv-shell .fv-index-scope button,
.fv-shell .fv-index-scope .widget-button button{
  background:linear-gradient(180deg,#22c55e 0%, #16a34a 100%) !important;
  color:#ffffff !important;
  border:0 !important;
  border-radius:14px !important;
  height:52px !important;
  padding:0 22px !important;
  font-weight:900 !important;
  font-size:15px !important;
  box-shadow:0 14px 28px rgba(34,197,94,.28) !important;
  cursor:pointer !important;
}

.fv-shell .fv-index-scope .is-clear button{
  background:#ffffff !important;
  color:#334155 !important;
  border:1px solid #e2e8f0 !important;
  box-shadow:0 10px 22px rgba(15,23,42,.06) !important;
}
</style>
"""))

# =================================================
# 2) Helpers
# =================================================
def get_screen(var_name, title):
    w = globals().get(var_name)
    if isinstance(w, widgets.Widget):
        return w

    return widgets.HTML(f"""
      <div style="background:#fff;border:1px solid #e5e7eb;
                  border-radius:14px;padding:24px;
                  box-shadow:0 10px 24px rgba(0,0,0,.06)">
        <h3 style="margin:0;color:#111827;font-size:18px;font-weight:900">{title}</h3>
        <p style="color:#6b7280;font-size:14px;margin-top:10px;line-height:1.6">
          Screen <b>{var_name}</b> not found.<br>
          Run the relevant cell that defines <code>{var_name}</code>.
        </p>
      </div>
    """)

def wrap_shell_screen(w):
    s = widgets.Box([w], layout=widgets.Layout(width="100%"))
    s.add_class("fv-shell-screen")
    return s

# =================================================
# 3) Screens (MUST match tabs order)
# =================================================
raw_screens = [
    get_screen("dashboard", "Dashboard"),
    get_screen("ui_screen", "Upload Image"),
    get_screen("index_screen", "Index search"),
    get_screen("rag_screen", "RAG"),
    get_screen("alerts_screen", "Alerts"),
]

screens = [wrap_shell_screen(s) for s in raw_screens]
screens_container = widgets.Box(screens, layout=widgets.Layout(width="100%"))

def show_only(idx: int):
    for i, s in enumerate(screens):
        s.layout.display = "block" if i == idx else "none"

def set_active_tab(i: int):
    for j, b in enumerate(tabs):
        if i == j:
            b.add_class("fv-active")
        else:
            try:
                b.remove_class("fv-active")
            except:
                pass

def go_to(idx: int):
    if idx < 0 or idx >= len(screens):
        print(f"Tab index {idx} has no screen. You have {len(screens)} screens.")
        return
    show_only(idx)
    set_active_tab(idx)

# Optional: go_to_name("rag")
TAB_NAME_TO_INDEX = {"dashboard": 0, "upload": 1, "index": 2, "rag": 3}
def go_to_name(name: str):
    name = (name or "").strip().lower()
    if name not in TAB_NAME_TO_INDEX:
        print("Unknown tab name:", name)
        return
    go_to(TAB_NAME_TO_INDEX[name])

# =================================================
# 4) Header + Tabs
# =================================================
header_top = widgets.HTML("""
<div class="fv-shell-header-top">
  <div class="fv-shell-logo-area">
    <div class="fv-shell-logo-box">🌱</div>
    <div>
      <div class="fv-shell-title">FloraVision</div>
      <div class="fv-shell-subtitle">Cloud-Based Precision Agriculture System</div>
    </div>
  </div>
  <div class="fv-shell-user-area">
    <div class="fv-shell-points">1250 Points</div>
    <div class="fv-shell-welcome">Welcome, Expert Farmer 🌱</div>
  </div>
</div>
""")

btn_overview = widgets.Button(description="Dashboard")
btn_alerts = widgets.Button(description="Alerts")
btn_upload   = widgets.Button(description="Upload Image")
btn_index    = widgets.Button(description="Index search")
btn_rag      = widgets.Button(description="RAG")

tabs = [btn_overview, btn_upload, btn_index, btn_rag, btn_alerts]

for b in tabs:
    b.add_class("fv-shell-tabbtn")

tabs_row = widgets.HBox(tabs)
tabs_row.add_class("fv-shell-tabs")

btn_overview.on_click(lambda _: go_to(0))
btn_upload.on_click(lambda _: go_to(1))
btn_index.on_click(lambda _: go_to(2))
btn_rag.on_click(lambda _: go_to(3))
btn_alerts.on_click(lambda _: go_to(4))



header = widgets.VBox([header_top, tabs_row])
header.add_class("fv-shell-header")

content = widgets.Box([screens_container])
content.add_class("fv-shell-content")

app = widgets.VBox([header, content])
app.add_class("fv-shell")

# Init
show_only(0)
set_active_tab(0)
display(app)

# expose API
globals()["go_to"] = go_to
globals()["go_to_name"] = go_to_name
globals()["screens"] = screens
globals()["screens_container"] = screens_container
globals()["tabs"] = tabs
